In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, scale

In [2]:
df = pd.read_csv('./baseline_data.csv',header=0,low_memory=False)

In [6]:
Catdata = df.loc[0:,'PTETHCAT':'APOE4'].join(df['PTGENDER']).join(df['imputed_genotype'])
for i in Catdata.columns:
    the_value = str(Catdata[i].mode().values[0]) # find the most common value in the column
    Catdata[i].replace('NaN',the_value,inplace = True)
    dummy_data = pd.get_dummies(Catdata[i], prefix=i+"_", drop_first=True) # https://www.jianshu.com/p/c324f4101785
    Catdata = pd.concat([Catdata, dummy_data], axis=1)
    Catdata.drop(i, axis=1, inplace=True)

Numdata = [];
feature_dis = 25
for i in range(feature_dis):
    Numdata.append(df.iloc[:,i:2150:feature_dis].join(df['AGE']).join(df['PTEDUCAT']).join(df['MMSE']))
#for i in Numdata[10].columns:
    #print(i)
scaler = StandardScaler()
data = Numdata[0].join(Catdata)
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data))
    
labels = df['DX.bl']
le = LabelEncoder()
le.fit(labels) # http://sofasofa.io/forum_main_post.php?postid=1001659
labels = le.transform(labels)
labels = labels.astype('float') # 1 CN, 0 AD, 2 LMCI


In [7]:
# only LogisticRegression
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

kfold = model_selection.KFold(n_splits=10, random_state=7)
cart = LogisticRegression()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=7)
results = model_selection.cross_val_score(model, data, labels, cv=kfold)
print(results.mean())

0.6320276497695853


In [11]:
# combine LogisticRegression， DecisionTree and SVM
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

seed=2
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
num_trees=50
estimators = []
lr = LogisticRegression()
model1 = BaggingClassifier(base_estimator=lr, n_estimators=num_trees, random_state=seed)
estimators.append(('logistic', model1))
cart = DecisionTreeClassifier()
model2 = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
estimators.append(('cart', model2))
svm = SVC()
model3 = BaggingClassifier(base_estimator=svm, n_estimators=num_trees, random_state=seed)
estimators.append(('svm', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, data, labels, cv=kfold)
print(results.mean())


D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.6735279057859704


D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
